<a target="_blank" href="https://colab.research.google.com/github/sonder-art/mhar/blob/released/nbs/tutorial_full_dimensional.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This tutorial will show how to use `mhar` for sampling polytopes. It is focused on executing parallel MCMC walks over a polytope in GPUs.

In [1]:
import torch

ModuleNotFoundError: No module named 'torch'

Before starting let's check if you have an avaialble gpu device or not.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').type
print('Device:', device)

Device: cuda


We also need to decide the data-type `dtype` we are going to use. Depending on your necessities you can choose it, we recomend to use `64` bits for non-fully dimentional polytopes in order to maintain numerical inestability of the projections. Otherwiise the precision depends on the dimension of your polytope and speed you want.  
  
As of now `16` bit precision is only available for `gpu` and not `cpu`.

In [ ]:
# We will choose 64-bits
dtype = torch.float64

## Canonical Representation

The polytope in question must be presented in matrix canonical representation (as opposed to vertex). `mhar` assumes that the matrix has no repeated or redundant restrictions.

## Fully dimensional Polytopes  

### Definition

> $A^IX \leq b^I$

For fully dimensional polytopes we need to use the class `Polytope` in the `mhar.polytope` module. The restrictions must be passed as pytorch tensors.  
  
We will sample the unit hypercube that is defined as:  
> $n-hypercube = \{x \in R^n || x \in [-1,1]^n \} $  

Which we can represent in matrix restrictions:  
$ Ix \leq 1$  
$ -Ix \leq 1$  
Where $I$ is the identity matrix of dimension $n \times n$ 


We will use this restrictions to define the polytope as:  
$A^Ix = [I | -I]x \leq 1 = b^I$

#### Definition-Code

Lets create the tensors to represent the restrictions that define the polytope.

In [ ]:
n = 3 # Dimension
dtype = torch.float32 # Precision 
A_I = torch.cat((torch.eye(n), torch.eye(n) * -1.0), dim=0).to(dtype) # Inequality Matrix
b_I = torch.ones(2 * n, dtype=dtype).view(-1, 1)  # Inequality restriction vector      
print(f'Inequality Matrix A^I \n {A_I} \n')
print(f'Inequality Vector b^I \n {b_I}')

Inequality Matrix A^I 
 tensor([[ 1.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 0.,  0.,  1.],
        [-1., -0., -0.],
        [-0., -1., -0.],
        [-0., -0., -1.]]) 

Inequality Vector b^I 
 tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])


Now lets create a `Polytope` object to represent the polytope.

In [ ]:
from mhar.polytope import Polytope
hypercube = Polytope(A_I, # Inequality Restriction Matrix 
                     b_I,  # Inequality Vector
                     dtype, # torch dtype
                     device, # device used cpu or cuda
                     copy=False # bool for creating a copy of the restrictions
                     )

/home/uumami/sonder.art/mhar/mhar/polytope.py:45: UserWarning:
  The object will not create a copy of the tensors, so modifications will be reflected in the object



In [ ]:
hypercube

Numeric Precision (dtype) torch.float32
Device: cuda
A_in: torch.Size([6, 3]) 
b_in: torch.Size([6, 1])

### Starting Inner Point(s)

In order to start the algorithm we need at least one inner point $x_0$. If you know your inner point you can supply it to the algorithm, `mhar` also contains functions to compute one inner point using the [chebyshev center](https://en.wikipedia.org/wiki/Chebyshev_center) which finds the center of the smallest ball inside the polytope.

 `from mhar.inner_point import ChebyshevCenter`. The solver is in numpy so precision must be specified as `numpy.dtype`. It uses `linprog` from `scipy.optimize`. You can see the documentation [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.linprog.html). 
  
It could also be the last points produced by a previous walk/run of the `mhar` 

In [ ]:
from mhar.inner_point import ChebyshevCenter
import numpy as np

In [ ]:
x0 = ChebyshevCenter(polytope=hypercube, # Polytope Object
                    lb=None,  # Lowerbound (lb <= x ), if unknown leave it as None 
                    ub=None,  # Upperbound ( x <= up), if unknown leave it as None 
                    tolerance=1e-4, # Tolerance for equality restrictions (A_eqx = b_eq)
                    device='cuda', # device used cpu or cuda
                    solver_precision=np.float32 # numpy dtype
                    )


Simplex Status for the Chebyshev Center
 Optimization proceeding nominally.


In [ ]:
x0

tensor([[-0.],
        [-0.],
        [-0.]], device='cuda:0')

If we want to manually input the inner points then it is enough to use a torch tensor of size $n \times l$. Where $l$ is ne number of inner points you want to supply. Just write them in column notation.  
  
We are going to manually add an other starting point to the one calcualted by the `chebyshev center` to show its functionality later.

In [ ]:
x0 = torch.cat([x0, 
            torch.tensor([[.5], [.5], [.5]]).to(device).to(dtype)
             ], dim=1)
x0

tensor([[-0.0000, 0.5000],
        [-0.0000, 0.5000],
        [-0.0000, 0.5000]], device='cuda:0')

Now we can proceed to sample the `polytope`

### Walk

We are going to sample the polytope starting from the inner points we supply using the method `walk.walk`. It has the next arguments:

+ `polytope` is an object of the type `Polytope` or `NFDPolytope` that defines it.
+ `X0` a tensor containing the inner points to start the walks from.
+ `z` determines the number of simoultaneous `walks`. If the number of initial points supplied are less than `z`  ($ncols($ `x0` $) < $ `z`) then some points will be reused as starting points.  
+ `T` is the number of uncorrelated iterations you want. The number of total uncorrelated points produced by the algorithm is `z` $\times$ `T`, since `z` points are sampled at each iteration.  
+ `thinning` determines the number of points that we need to burn between iterations in order to get uncorrelated points. The suggested factor should be in the order of $O(n^3)$.
+ `warm` determines a thinning for warming the walks only at the beggining, after the this war the walks resumes as normal. It is used if you want to lose the dependency from the starting points.
+ `device` device where the tenros live `cpu` or `cuda`
+ `seed` for reproducibility
+ `verbosity` for printing what is going on

In [ ]:
from mhar.walk import walk
X = walk(polytope=hypercube,
        X0 = x0,  
        z=100, 
        T=1, 
        warm=0,
        thinning=3**3, 
        device= None, 
        seed=None,
        verbosity=2
)

Minimum number allowed -3.4028234663852886e+38
Maximum number allowed 3.4028234663852886e+38
Eps:  1.1920928955078125e-07
Values close to zero will be converted to 3eps or -3eps: 3.5762786865234375e-07
n:  3   mI: 6   mE: None   z: 100
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |██████████████████████████████| 100.0%


`walk` produces `T` $\times$ `z` uncorrelated points. It returns a vector of dimension `T` $\times$ `z` $\times$ `n`.  

In [ ]:
X

tensor([[[ 0.1346,  0.1896,  0.8120,  0.2621, -0.1261, -0.6093, -0.8195,
           0.4585, -0.6270,  0.2019,  0.2617, -0.2022, -0.6854, -0.8556,
          -0.2954, -0.4730, -0.8601, -0.3819, -0.9203, -0.7145,  0.2585,
           0.3860, -0.3928, -0.0756,  0.7155, -0.0701, -0.6295, -0.9855,
          -0.4843,  0.6900, -0.1447, -0.7108, -0.5022, -0.9553,  0.3655,
          -0.3814,  0.5704,  0.6801,  0.2533,  0.9957,  0.0294,  0.5533,
           0.1137, -0.9384, -0.3501,  0.0382, -0.1064,  0.8821,  0.9229,
          -0.5929, -0.1313,  0.8912, -0.5393, -0.2716, -0.4064, -0.6508,
          -0.2260,  0.7822,  0.1814,  0.1944, -0.3762, -0.5079,  0.1859,
          -0.0317,  0.0774, -0.7498, -0.6107, -0.9015,  0.5549,  0.9456,
           0.7679, -0.4838,  0.6325, -0.5565,  0.5174,  0.4736, -0.2022,
          -0.4906, -0.9142, -0.2539, -0.7834, -0.9992,  0.8694,  0.5327,
           0.0498, -0.0449, -0.4333, -0.1756, -0.9705, -0.0613,  0.4504,
           0.1786,  0.8194,  0.9220, -0.2853, -0.96

### Summary

To sumamrize the steps taken we can use the `polytope_examples` for creating a `Hypercube`.

In [ ]:
from mhar.polytope_examples import Hypercube

# Create a polytope (Hypercube)
hypercube = Hypercube(10,
                      dtype=torch.float32,
                      device='cuda'
                      )

/home/uumami/sonder.art/mhar/mhar/polytope.py:45: UserWarning:
  The object will not create a copy of the tensors, so modifications will be reflected in the object



Define/Find inner points

In [ ]:
x0 = ChebyshevCenter(polytope=hypercube, 
                    lb=None, 
                    ub=None, 
                    tolerance=1e-4,
                    device='cuda',
                    solver_precision=np.float32)




Simplex Status for the Chebyshev Center
 Optimization proceeding nominally.


Sample points

In [ ]:
X = walk(polytope=hypercube,
        X0 = x0,  
        z=100, 
        T=1, 
        warm=0,
        thinning=10000, 
        device= None, 
        seed=None,
        verbosity=2
)
X

Minimum number allowed -3.4028234663852886e+38
Maximum number allowed 3.4028234663852886e+38
Eps:  1.1920928955078125e-07
Values close to zero will be converted to 3eps or -3eps: 3.5762786865234375e-07
n:  10   mI: 20   mE: None   z: 100
% of burned samples |██████████████████████████████| 100.0%
% of iid samples |██████████████████████████████| 100.0%


tensor([[[-3.8327e-01,  7.8069e-02,  2.9651e-01,  6.9654e-01, -9.8001e-01,
          -7.6946e-01,  9.5175e-01, -3.8984e-01, -2.3625e-01,  3.4791e-02,
           9.0714e-01,  1.2673e-01,  6.5063e-01,  3.3621e-01, -4.3635e-01,
          -5.0896e-01, -9.7254e-01, -5.6308e-01,  1.9761e-01,  3.7554e-01,
           6.2283e-01, -6.3949e-01, -1.2451e-02, -6.9506e-01, -6.0431e-01,
          -4.7805e-01, -5.9062e-01,  3.3656e-01,  3.5997e-01, -7.2770e-01,
           7.9037e-01, -6.7223e-01, -2.8605e-01, -5.9226e-01,  4.7954e-01,
           7.4920e-02,  7.0809e-01, -3.3553e-01,  9.6565e-01, -2.4111e-01,
           2.0464e-01,  4.4271e-01,  3.8480e-01, -1.0596e-03,  3.6338e-01,
          -9.3107e-01,  5.2171e-01, -7.4777e-01, -9.5047e-02, -6.4247e-01,
          -2.5327e-02, -1.5993e-01,  6.2042e-01, -3.8698e-01,  3.5689e-01,
          -5.2772e-01, -8.7221e-01, -4.0191e-01, -5.3989e-01,  8.4517e-01,
           5.2379e-01, -6.2028e-01,  7.3326e-02, -7.7204e-01, -4.3243e-01,
          -9.5900e-01,  1